Import modules for our purpose

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

Get the page url and test if its response correctly

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_html = requests.get(url)
page_html

<Response [200]>

Create a BeautifulSoup object

In [3]:
page_obj = BeautifulSoup(page_html.content,"html5lib")
page_obj.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   List of postal codes of Canada: M - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"2a24c97b-fc5f-46be-a834-1c98b95da663","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","

In [4]:
table = page_obj.find("table")# find the table 
content=[] 
for row in table.findAll('td'): # for on every td
    cell={}
    if row.span.text == 'Not assigned':#Not assigned filter
            pass
    else:
        cell['Postal Code'] = row.p.text[:3] # Get first 3 ch of ever td
        cell['Borough'] = row.p.span.text.split('(')[0] # Get all string before a '(' in the span
        cell['Neighborhood'] = row.p.span.text.split('(')[1].replace('/',',').replace(')','') # Get all string after a '(' in the span and replace wrong ch
        content.append(cell)
        
df = pd.DataFrame(content)# create pandas df
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


Let's see every Borough

In [5]:
df['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East Toronto                                                     4
East York                                                        4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
Queen's Park                                                     1
MississaugaCanada Post Gateway Processing Centre                 1
East YorkEast Toronto                                            1
EtobicokeNorthwest                                               1
East TorontoBusiness reply mail Processing Centre969 Eastern  

Some 1 hits value can be replace with correct one

In [6]:
df['Borough'].replace({'East YorkEast Toronto':'East York / East Toronto',
                       'MississaugaCanada Post Gateway Processing Centre': 'Missisauga',
                      'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto',
                      'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto',
                      'EtobicokeNorthwest':'Etobicoke'},inplace=True)

df['Borough'].value_counts()

North York                  24
Downtown Toronto            18
Scarborough                 17
Etobicoke                   12
Central Toronto              9
West Toronto                 6
York                         5
East Toronto                 5
East York                    4
East York / East Toronto     1
Missisauga                   1
Queen's Park                 1
Name: Borough, dtype: int64

In [7]:
df.shape

(103, 3)

Let's get coordinates

In [8]:
!pip install geocoder

In [9]:
import geocoder

In [10]:
# for pc in df['Postal Code']:
#     print(pc)
#     get_lat_lon = None
#     while (get_lat_lon is None):
#         coordinates = geocoder.google('Toronto, ON {}'.format(pc))
#         get_lat_lon = coordinates.latlng
#     df['Latitude'] = get_lat_lon[0]
#     df['Longitude'] = get_lat_lon[1]
    
# df

Bad news from the geocoder :( Let's use provide csv!

In [11]:
gc = pd.read_csv('Geospatial_Coordinates.csv')
gc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
df = pd.merge(df,gc, on='Postal Code')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


Print Map of Toronto with neighborhood

In [13]:
#!conda install -c conda-forge folium=0.11.0 --yes 

In [14]:
import folium

In [15]:
latitude = 43.72732
longitude =  -79.384955
def get_toronto_map (df):
    toronto_map = folium.Map(location=[latitude,longitude],zoom_start=11,tiles='OpenStreetMap')
    for lat, lon, bor, neigh in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
        label = '{}, {}'.format(neigh, bor)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 
    return toronto_map

toronto_map = get_toronto_map(df)
toronto_map

Get only Toronto Borough

In [16]:
t = df[df['Borough'].str.contains('Toronto',na=False)]
t

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
35,M4J,East York / East Toronto,The Danforth East,43.685347,-79.338106


In [17]:
toronto_map = get_toronto_map(t)
toronto_map

In [18]:
CLIENT_ID ='KEAE0PZEAU05ECG52IV3OVDF1PHT0TIPKMFW3XBS2WODWMJO'
CLIENT_SECRET ='MSCZOARQL1VCZPQBSXOA3WL5AAK4QRJTHEIDV51SWO3N23ST'
VERSION = '20180605'
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500): #function to get all nearby venues (from prev notebook)
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names = t['Neighborhood'],
                                latitudes=t['Latitude'],
                                longitudes=t['Longitude'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's Explore the different type of Venue

In [20]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop              140
Café                      83
Restaurant                53
Bakery                    40
Hotel                     39
                        ... 
Stadium                    1
Building                   1
Flower Shop                1
Sri Lankan Restaurant      1
Tibetan Restaurant         1
Name: Venue Category, Length: 232, dtype: int64

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="",prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']


toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower , King and Spadina , Railway Lands , ...",0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.014925,0.000000,0.000000,0.000000,0.014925
4,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013158,0.000000,0.00,...,0.013158,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.026316
6,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.010000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin , Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


Print of neighborhoods with them top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.05
1  Cocktail Bar  0.05
2        Bakery  0.05
3      Pharmacy  0.04
4      Beer Bar  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3          Bakery  0.04
4       Pet Store  0.04


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3           Airport  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Place  0.04
3  Italian Restaurant  0.04
4          Restaurant  0.03


----Christie----
                venue  freq
0       Grocery Store  0.27
1                Café  0.20
2                Park  0.13
3         Candy Store  0.07
4  Italian R

In [23]:
def return_most_common_venues(row, num_top_venues): # from prev notebook
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
top = 10

indicators =['st','nd','rd']

columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most common Venue'.format(ind+1),indicators[ind])
    except:
        columns.append('{}th Most common Venue'.format(ind+1))
        
neigh_venues = pd.DataFrame(columns=columns)
neigh_venues['Neighborhood'] = toronto_grouped['Neighborhood']


for ind in np.arange(toronto_grouped.shape[0]):
    neigh_venues.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
    
neigh_venues

,Neighborhood,1th Most common Venue,2th Most common Venue,3th Most common Venue,4th Most common Venue,5th Most common Venue
0,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Pharmacy,Cheese Shop
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden
3,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant
4,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant
5,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
6,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym
7,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Café,Coffee Shop
8,Davisville North,Park,Breakfast Spot,Pizza Place,Department Store,Sandwich Place
9,"Dufferin , Dovercourt Village",Bakery,Pharmacy,Grocery Store,Music Venue,Supermarket


Clusters!

In [25]:
from sklearn.cluster import KMeans

In [26]:
ks = 5
toronto_clustering = toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=ks,random_state=0).fit(toronto_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [27]:
neigh_venues.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = t

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neigh_venues.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

toronto_merged.dropna(axis=0,inplace=True)

In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i + x + (i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most common Venue,2th Most common Venue,3th Most common Venue,4th Most common Venue,5th Most common Venue
25,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant
83,Central Toronto,0,Park,Tennis Court,Restaurant,Diner,Falafel Restaurant


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most common Venue,2th Most common Venue,3th Most common Venue,4th Most common Venue,5th Most common Venue
2,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Hotel
15,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Clothing Store
19,East Toronto,1,Health Food Store,Trail,Pub,Yoga Studio,Discount Store
20,Downtown Toronto,1,Cocktail Bar,Bakery,Coffee Shop,Pharmacy,Cheese Shop
24,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant
30,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Restaurant,Hotel
31,West Toronto,1,Bakery,Pharmacy,Grocery Store,Music Venue,Supermarket
36,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Restaurant
37,West Toronto,1,Bar,Restaurant,Vietnamese Restaurant,Diner,Men's Store
